In [ ]:
# comment out this cell if not running on colab
from google.colab import drive
drive.mount('/content/drive')

# copy zip to local directory
print("Copying dataset from Google Drive...")
!cp "/content/drive/My Drive/datasets/top50images.zip" .

print("Unzipping dataset...")
!unzip -q top50images.zip

Mounted at /content/drive
Copying dataset from Google Drive...
Unzipping dataset...


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models
from torch.utils.data import DataLoader, random_split, WeightedRandomSampler, Dataset
import pandas as pd
import time
import os
from torchvision import transforms
from PIL import Image

In [ ]:
# paths to dataset and csv
CSV_PATH = 'top50.csv'
IMAGE_FOLDER_PATH = 'top50images'

In [ ]:
class CityImageDataset(Dataset):
    """Custom PyTorch Dataset for city image coordinate regression."""
    def __init__(self, csv_path, image_folder, transform=None):
        """
        Args:
            csv_path (str): Path to the csv file with annotations.
            image_folder (str): Directory with all the images.
            transform (callable, optional): Optional transform to be applied on a sample.
        """
        self.img_labels = pd.read_csv(csv_path)
        self.image_folder = image_folder
        self.transform = transform

        # For regression, we don't need class-to-index mapping.
        print("Dataset initialized for regression.")

    def __len__(self):
        """Returns the total number of samples in the dataset."""
        return len(self.img_labels)

    def __getitem__(self, idx):
        """
        Fetches a data sample for a given index.

        Returns:
            tuple: (image, coordinates) where coordinates is a tensor of [lat, lon].
        """
        # 1. Get the image filename and coordinate labels from the dataframe
        row = self.img_labels.iloc[idx]
        image_filename = row['filename']
        lat = row['lat']
        lon = row['lon']

        # 2. Construct the full image path
        image_path = os.path.join(self.image_folder, image_filename)
        image = Image.open(image_path).convert('RGB')

        # 3. Create the label tensor for regression
        coordinates = torch.tensor([lat, lon], dtype=torch.float32)

        # 4. Apply transformations if they exist
        if self.transform:
            image = self.transform(image)

        return image, coordinates

In [ ]:
# transform data for resnet
data_transforms = transforms.Compose([
    transforms.Resize((224, 224)),      # resize to 224 x 224
    transforms.ToTensor(),
    transforms.Normalize(               # normalize to imagenet stats
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

# create dataset
print("Creating dataset...")
full_dataset = CityImageDataset(
    csv_path=CSV_PATH,
    image_folder=IMAGE_FOLDER_PATH,
    transform=data_transforms
)

# split data into 80/20 for train/val
dataset_size = len(full_dataset)
train_size = int(dataset_size * 0.8)
val_size = dataset_size - train_size

Creating dataset...
Dataset initialized for regression.


In [ ]:
# seed and split
generator = torch.Generator().manual_seed(42)
train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size], generator=generator)

print(f"Total samples: {dataset_size}")
print(f"Training samples: {len(train_dataset)}")
print(f"Validation samples: {len(val_dataset)}")

Total samples: 8023
Training samples: 6418
Validation samples: 1605


In [ ]:
# create data loaders
BATCH_SIZE = 32
train_loader = DataLoader(
    dataset=train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=2) # change this to 0 if running locally (not colab)

val_loader = DataLoader(
    dataset=val_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=2) # change this to 0 if running locally (not colab)

/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


In [ ]:
# get device
if torch.cuda.is_available():
    device = torch.device("cuda")
elif torch.backends.mps.is_available():
    device = torch.device("mps")
else:
    device = torch.device("cpu")
print(f"Using device: {device}")

Using device: cuda


In [ ]:
# load resnet model
resnet50 = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V2)
# model = models.resnet50(weights='DEFAULT')

# freeze params/layers of model
print("Freezing parameters of pre-trained layers...")
for param in resnet50.parameters():
    param.requires_grad = False

# replace the final layer for regression. output is 2 for lat and lon
num_ftrs = resnet50.fc.in_features
resnet50.fc = nn.Linear(num_ftrs, 2)
print(f"Replaced final layer for regression. Output features: 2")

Freezing parameters of pre-trained layers...
Replaced final layer for regression. Output features: 2


In [ ]:
# move model to device
resnet50 = resnet50.to(device)

In [ ]:
# hyperparams
NUM_EPOCHS = 60
LEARNING_RATE = 0.001

In [ ]:
# set up loss func and optimizer

# loss func
loss_func = nn.MSELoss()

# will only update weights for final layer since we froze previous ones
optimizer = optim.Adam(resnet50.fc.parameters(), lr=LEARNING_RATE)

In [ ]:
def train_and_validate(model, loss_func, optimizer, train_loader, val_loader, device, num_epochs=5):
    """
    Args:
        model: model to train
        loss_func: loss function
        optimizer: optimizer
        train_loader: DataLoader for the training set
        val_loader: DataLoader for the validation set
        device: the torch.device to run on
        num_epochs: number of epochs
    """
    print()
    print("--- Starting Training ---")
    for epoch in range(num_epochs):

        # keep track of how much time each epoch takes
        start_time = time.time()

        # TRAINING
        model.train()  # set to training mode
        running_loss = 0.0  # keep track of loss

        # iterate through batches
        for images, labels in train_loader:

            # move to device
            images, labels = images.to(device), labels.to(device)

            # zero out gradients from previous batch
            optimizer.zero_grad()

            # forward pass
            outputs = model(images)

            # apply loss function
            loss = loss_func(outputs, labels)

            # backprop
            loss.backward()

            # update weights
            optimizer.step()

            # keep track of loss
            running_loss += loss.item() * images.size(0)

        # keep track of total epoch loss
        epoch_train_loss = running_loss / len(train_loader.sampler)

        # VALIDATION
        model.eval()  # set to eval mode
        val_loss = 0.0
        correct_predictions = 0

        # no gradients
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)

                # apply forward pass and evaluate output
                outputs = model(images)
                loss = loss_func(outputs, labels)
                val_loss += loss.item() * images.size(0)

        # keep track of total epoch val loss
        epoch_val_loss = val_loss / len(val_loader.dataset)

        # time it took for epoch to train
        epoch_time = time.time() - start_time
        print(f"Epoch {epoch + 1}/{num_epochs} | Time: {epoch_time:.2f}s")
        print(f"\t Train Loss: {epoch_train_loss:.4f} | Val Loss: {epoch_val_loss:.4f}")

    print()
    print("--- Training Complete ---")


In [ ]:
train_and_validate(resnet50, loss_func, optimizer, train_loader, val_loader, device, NUM_EPOCHS)


--- Starting Training ---
Epoch 1/60 | Time: 44.00s
	 Train Loss: 3435.3758 | Val Loss: 1967.1636
Epoch 2/60 | Time: 44.19s
	 Train Loss: 1196.3921 | Val Loss: 874.5019
Epoch 3/60 | Time: 42.59s
	 Train Loss: 518.3372 | Val Loss: 451.5892
Epoch 4/60 | Time: 43.07s
	 Train Loss: 366.2274 | Val Loss: 338.5996
Epoch 5/60 | Time: 42.59s
	 Train Loss: 335.5985 | Val Loss: 312.4147
Epoch 6/60 | Time: 42.42s
	 Train Loss: 325.2496 | Val Loss: 317.7475
Epoch 7/60 | Time: 42.77s
	 Train Loss: 309.1009 | Val Loss: 297.3292
Epoch 8/60 | Time: 42.64s
	 Train Loss: 306.8656 | Val Loss: 305.2011
Epoch 9/60 | Time: 42.60s
	 Train Loss: 298.8620 | Val Loss: 276.6658
Epoch 10/60 | Time: 42.52s
	 Train Loss: 290.1457 | Val Loss: 272.0964
Epoch 11/60 | Time: 42.72s
	 Train Loss: 273.1789 | Val Loss: 267.3201
Epoch 12/60 | Time: 42.77s
	 Train Loss: 273.7532 | Val Loss: 263.5162
Epoch 13/60 | Time: 42.99s
	 Train Loss: 265.9641 | Val Loss: 257.4835
Epoch 14/60 | Time: 43.18s
	 Train Loss: 257.4150 | Val 

In [ ]:
# one degree of latitude -> roughly 69 miles (111 km)
#

In [ ]:

# --- Starting Training ---
# Epoch 1/60 | Time: 44.00s
# 	 Train Loss: 3435.3758 | Val Loss: 1967.1636
# Epoch 2/60 | Time: 44.19s
# 	 Train Loss: 1196.3921 | Val Loss: 874.5019
# Epoch 3/60 | Time: 42.59s
# 	 Train Loss: 518.3372 | Val Loss: 451.5892
# Epoch 4/60 | Time: 43.07s
# 	 Train Loss: 366.2274 | Val Loss: 338.5996
# Epoch 5/60 | Time: 42.59s
# 	 Train Loss: 335.5985 | Val Loss: 312.4147
# Epoch 6/60 | Time: 42.42s
# 	 Train Loss: 325.2496 | Val Loss: 317.7475
# Epoch 7/60 | Time: 42.77s
# 	 Train Loss: 309.1009 | Val Loss: 297.3292
# Epoch 8/60 | Time: 42.64s
# 	 Train Loss: 306.8656 | Val Loss: 305.2011
# Epoch 9/60 | Time: 42.60s
# 	 Train Loss: 298.8620 | Val Loss: 276.6658
# Epoch 10/60 | Time: 42.52s
# 	 Train Loss: 290.1457 | Val Loss: 272.0964
# Epoch 11/60 | Time: 42.72s
# 	 Train Loss: 273.1789 | Val Loss: 267.3201
# Epoch 12/60 | Time: 42.77s
# 	 Train Loss: 273.7532 | Val Loss: 263.5162
# Epoch 13/60 | Time: 42.99s
# 	 Train Loss: 265.9641 | Val Loss: 257.4835
# Epoch 14/60 | Time: 43.18s
# 	 Train Loss: 257.4150 | Val Loss: 248.1742
# Epoch 15/60 | Time: 42.92s
# 	 Train Loss: 249.9022 | Val Loss: 246.6748
# Epoch 16/60 | Time: 43.49s
# 	 Train Loss: 249.1971 | Val Loss: 258.5792
# Epoch 17/60 | Time: 43.66s
# 	 Train Loss: 243.4445 | Val Loss: 244.2865
# Epoch 18/60 | Time: 43.74s
# 	 Train Loss: 234.6058 | Val Loss: 230.3819
# Epoch 19/60 | Time: 43.25s
# 	 Train Loss: 232.8695 | Val Loss: 225.3836
# Epoch 20/60 | Time: 42.88s
# 	 Train Loss: 226.6339 | Val Loss: 220.6066
# Epoch 21/60 | Time: 42.52s
# 	 Train Loss: 219.0096 | Val Loss: 223.8359
# Epoch 22/60 | Time: 42.61s
# 	 Train Loss: 217.8381 | Val Loss: 217.9048
# Epoch 23/60 | Time: 42.57s
# 	 Train Loss: 209.5956 | Val Loss: 216.2779
# Epoch 24/60 | Time: 42.98s
# 	 Train Loss: 206.5770 | Val Loss: 210.6333
# Epoch 25/60 | Time: 42.32s
# 	 Train Loss: 203.4308 | Val Loss: 208.5687
# Epoch 26/60 | Time: 42.70s
# 	 Train Loss: 201.6012 | Val Loss: 207.7472
# Epoch 27/60 | Time: 43.01s
# 	 Train Loss: 199.7480 | Val Loss: 202.2208
# Epoch 28/60 | Time: 42.58s
# 	 Train Loss: 197.9698 | Val Loss: 212.3949
# Epoch 29/60 | Time: 42.00s
# 	 Train Loss: 192.6947 | Val Loss: 202.5450
# Epoch 30/60 | Time: 42.54s
# 	 Train Loss: 190.6640 | Val Loss: 202.7761
# Epoch 31/60 | Time: 42.46s
# 	 Train Loss: 186.4895 | Val Loss: 194.0509
# Epoch 32/60 | Time: 42.87s
# 	 Train Loss: 187.0362 | Val Loss: 193.7160
# Epoch 33/60 | Time: 43.15s
# 	 Train Loss: 179.9695 | Val Loss: 193.8890
# Epoch 34/60 | Time: 43.87s
# 	 Train Loss: 180.9153 | Val Loss: 193.4415
# Epoch 35/60 | Time: 44.21s
# 	 Train Loss: 176.2423 | Val Loss: 191.9749
# Epoch 36/60 | Time: 42.74s
# 	 Train Loss: 176.4518 | Val Loss: 188.5550
# Epoch 37/60 | Time: 43.00s
# 	 Train Loss: 175.1528 | Val Loss: 185.5869
# Epoch 38/60 | Time: 44.32s
# 	 Train Loss: 170.0210 | Val Loss: 188.2923
# Epoch 39/60 | Time: 42.73s
# 	 Train Loss: 172.9848 | Val Loss: 190.6229
# Epoch 40/60 | Time: 42.44s
# 	 Train Loss: 165.2481 | Val Loss: 181.8752
# Epoch 41/60 | Time: 44.46s
# 	 Train Loss: 171.6007 | Val Loss: 195.2496
# Epoch 42/60 | Time: 44.21s
# 	 Train Loss: 169.9812 | Val Loss: 180.3747
# Epoch 43/60 | Time: 43.16s
# 	 Train Loss: 163.8982 | Val Loss: 184.7925
# Epoch 44/60 | Time: 45.12s
# 	 Train Loss: 162.3506 | Val Loss: 178.5508
# Epoch 45/60 | Time: 42.93s
# 	 Train Loss: 162.2778 | Val Loss: 180.4169
# Epoch 46/60 | Time: 42.83s
# 	 Train Loss: 160.2674 | Val Loss: 177.7327
# Epoch 47/60 | Time: 42.84s
# 	 Train Loss: 160.6646 | Val Loss: 174.8604
# Epoch 48/60 | Time: 45.63s
# 	 Train Loss: 154.2337 | Val Loss: 179.5932
# Epoch 49/60 | Time: 42.61s
# 	 Train Loss: 156.3259 | Val Loss: 193.1016
# Epoch 50/60 | Time: 42.26s
# 	 Train Loss: 155.0227 | Val Loss: 173.0665
# Epoch 51/60 | Time: 44.91s
# 	 Train Loss: 156.9886 | Val Loss: 175.3542
# Epoch 52/60 | Time: 42.20s
# 	 Train Loss: 157.5791 | Val Loss: 173.6359
# Epoch 53/60 | Time: 42.60s
# 	 Train Loss: 154.4936 | Val Loss: 172.7473
# Epoch 54/60 | Time: 44.88s
# 	 Train Loss: 151.9312 | Val Loss: 175.2835
# Epoch 55/60 | Time: 42.95s
# 	 Train Loss: 151.7344 | Val Loss: 179.1439
# Epoch 56/60 | Time: 42.43s
# 	 Train Loss: 148.9095 | Val Loss: 173.0782
# Epoch 57/60 | Time: 43.39s
# 	 Train Loss: 150.3910 | Val Loss: 171.1948
# Epoch 58/60 | Time: 43.28s
# 	 Train Loss: 148.9218 | Val Loss: 168.1417
# Epoch 59/60 | Time: 43.29s
# 	 Train Loss: 147.2358 | Val Loss: 170.3907
# Epoch 60/60 | Time: 42.66s
# 	 Train Loss: 148.1556 | Val Loss: 170.4782

# --- Training Complete ---